<a href="https://colab.research.google.com/github/Nishigandha-Wankhade/One-to-Many-Text-Generation-Using-RNN-50-Epochs-/blob/main/RNN_one_to_many_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project 4: One-to-Many Text Generation Using RNN (50 Epochs)

In [13]:
# Step 1: Import libraries and set constants
import tensorflow as tf
import numpy as np
import os

In [14]:
# Set constants
SEQ_LENGTH = 100
BATCH_SIZE = 64
BUFFER_SIZE = 10000
EMBEDDING_DIM = 256
RNN_UNITS = 1024
EPOCHS = 50

In [15]:
# Step 2: Download and prepare text data (Shakespeare text example)
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')


Length of text: 1115394 characters


In [16]:
# Step 3: Create the vocabulary
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [17]:
# Step 4: Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)


In [18]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [19]:
dataset = sequences.map(split_input_target)

In [20]:
# Step 5: Batch and shuffle
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=False, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(len(vocab), EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)


In [23]:
# Step 7: Define loss and compile the model
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)


In [ ]:
# Step 8: Train the model for 50 epochs
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 1418s 8s/step - loss: 2.9130
Epoch 2/50
110/172 ━━━━━━━━━━━━━━━━━━━━ 8:19 8s/step - loss: 1.9028